In [2]:
!pip install econml

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.3 MB 684.7 kB/s eta 0:00:03
   ------------- -------------------------- 0.8/2.3 MB 788.8 kB/s eta 0:00:02
   ------------- -------------------------- 0.8/2.3 MB 788.8 kB/s eta 0:00:02
   ------------------ --------------------- 1.0/2.3 MB 770.0 kB/s eta 0:00:02
   ------------------ --------------------- 1.0/2.3 MB 770.0 kB/s eta 0:00:02
   ----------------------- ---------------- 1.3/2.3 MB 791.8 kB/s eta 0:00:02
   --------------------------- ------------ 1.6/2.3 MB 801.2 kB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.3 MB 801.2 kB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.3 MB 801.2 kB/s eta 0:00:01
   -------------------

  You can safely remove it manually.
  You can safely remove it manually.


In [3]:
import numpy as np
import pandas as pd
from econml.dml import DML
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split


In [9]:
from econml.dml import DML
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import numpy as np

# -----------------------------------------------------
# 1. Generate synthetic dataset
# -----------------------------------------------------
np.random.seed(42)
n = 2000

X = np.random.normal(0, 1, size=(n, 5))
T = (X[:, 0] + X[:, 1] + np.random.randn(n) > 0).astype(int)  # binary treatment
Y = 3*T + X[:, 2] + np.random.randn(n)

# -----------------------------------------------------
# 2. Models for nuisance (BOTH must be regressors)
# -----------------------------------------------------
model_y = RandomForestRegressor()
model_t = RandomForestRegressor()      # IMPORTANT: changed to regressor

# final stage model
model_final = LinearRegression()

# -----------------------------------------------------
# 3. DML estimator
# -----------------------------------------------------
dml = DML(
    model_y=model_y,
    model_t=model_t,
    model_final=model_final,
    random_state=42
)

# -----------------------------------------------------
# 4. FIT — use keyword arguments
# -----------------------------------------------------
dml.fit(Y=Y, T=T, X=X)

# -----------------------------------------------------
# 5. ATE
# -----------------------------------------------------
ate = dml.ate(X)
print("Estimated ATE:", ate)


Estimated ATE: 2.854488934404438


C:\Users\ASUS\anaconda3\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "


In [10]:
# dml_pipeline_synthetic_improved.py
# Run with: python dml_pipeline_synthetic_improved.py
# Requires: numpy, pandas, scikit-learn, statsmodels, matplotlib, seaborn
# pip install numpy pandas scikit-learn statsmodels matplotlib seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.model_selection import KFold, train_test_split
import statsmodels.api as sm
import time
from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

# reproducibility
SEED = 42
np.random.seed(SEED)

# ---------- SETTINGS ----------
n = 5000           # sample size (reduce if needed)
p = 60             # number of features (>=50)
K = 5              # cross-fitting folds
noise_scale = 1.0
n_estimators = 80
max_depth = 4
BOOTSTRAP_B = 500   # bootstrap iterations for CI
SAVE_PREFIX = "dml_run"
# ------------------------------

# ----------------
# 1) Synthetic data generation (complex non-linear relationships)
# ----------------
def sigmoid(x): return 1 / (1 + np.exp(-x))

X = np.random.normal(size=(n, p))
propensity_score = (
    0.8 * np.tanh(X[:, 0] * 1.2 + X[:, 1]**2 - 0.5 * X[:, 2]) +
    0.6 * np.sin(0.5 * X[:, 3] * X[:, 4]) +
    0.4 * (X[:, 5] * X[:, 6]) +
    0.3 * (X[:, 7]**3) +
    0.2 * np.sum(X[:, 8:12] * np.linspace(0.1, 0.4, 4), axis=1)
)
propensity = sigmoid(propensity_score - np.median(propensity_score))
propensity = np.clip(propensity, 0.02, 0.98)
T = np.random.binomial(1, propensity, size=n)

true_tau = 2.0 * sigmoid(0.6 * X[:, 0] + 0.4 * X[:, 1]**2 - 0.3 * X[:, 2])

mu_X = (
    1.5 * np.cos(X[:, 0]) +
    0.8 * (X[:, 1]**2) -
    0.6 * np.tanh(X[:, 2] * X[:, 3]) +
    0.5 * X[:, 4] +
    0.3 * X[:, 5] * X[:, 6] +
    0.2 * np.sum(X[:, 10:18] * np.linspace(0.05, 0.2, 8), axis=1)
)

Y = mu_X + T * true_tau + np.random.normal(scale=noise_scale, size=n)

colnames = [f"X{i}" for i in range(p)]
df = pd.DataFrame(X, columns=colnames)
df['T'] = T
df['Y'] = Y
df['propensity_true'] = propensity
df['tau_true'] = true_tau

print(f"Generated dataset n={n}, p={p}, mean(T)={T.mean():.3f}")

# ----------------
# 2) Quick EDA / checks (positivity & overlap)
# ----------------
plt.figure(figsize=(6,4))
sns.histplot(df['propensity_true'], bins=40, kde=False)
plt.title("True propensity score distribution")
plt.xlabel("propensity_true")
plt.tight_layout()
plt.savefig(f"{SAVE_PREFIX}_propensity_true_hist.png", dpi=150)
plt.close()

# ----------------
# 3) Cross-fitting to get out-of-fold nuisance predictions
# ----------------
kf = KFold(n_splits=K, shuffle=True, random_state=SEED)
m_hat = np.zeros(n)   # E[Y|X]
g_hat = np.zeros(n)   # E[T|X] (propensity)
model_m_importances = []
model_g_importances = []

start = time.time()
for fold, (train_idx, test_idx) in enumerate(kf.split(df), 1):
    X_train = df.iloc[train_idx][colnames].values
    X_test = df.iloc[test_idx][colnames].values
    Y_train = df.iloc[train_idx]['Y'].values
    T_train = df.iloc[train_idx]['T'].values

    # outcome model (regressor)
    model_m = HistGradientBoostingRegressor(max_iter=n_estimators, max_depth=max_depth, random_state=SEED+fold)
    model_m.fit(X_train, Y_train)
    m_hat[test_idx] = model_m.predict(X_test)

    # treatment model (classifier for propensity; we will use predict_proba)
    model_g = HistGradientBoostingClassifier(max_iter=n_estimators, max_depth=max_depth, random_state=SEED+fold)
    model_g.fit(X_train, T_train)
    # predict_proba -> probability of T=1
    g_hat[test_idx] = model_g.predict_proba(X_test)[:, 1]

    # record permutation importances on test to report later (small cost)
    try:
        imp_m = permutation_importance(model_m, X_test, df.iloc[test_idx]['Y'].values, n_repeats=10, random_state=SEED)
        model_m_importances.append(imp_m.importances_mean)
    except Exception:
        model_m_importances.append(np.zeros(len(colnames)))
    try:
        imp_g = permutation_importance(model_g, X_test, df.iloc[test_idx]['T'].values, n_repeats=10, random_state=SEED)
        model_g_importances.append(imp_g.importances_mean)
    except Exception:
        model_g_importances.append(np.zeros(len(colnames)))

print(f"Cross-fitting finished in {time.time() - start:.1f}s")

# ----------------
# 4) Diagnostics for nuisance fits
# ----------------
# Propensity calibration plot
prob_true, prob_pred = calibration_curve(df['T'].values, g_hat, n_bins=10)
plt.figure(figsize=(6,6))
plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1],'--', alpha=0.6)
plt.xlabel("Predicted propensity")
plt.ylabel("Observed frequency")
plt.title("Propensity calibration (oof g_hat)")
plt.tight_layout()
plt.savefig(f"{SAVE_PREFIX}_propensity_calibration.png", dpi=150)
plt.close()

# Propensity AUC (informative)
try:
    auc_oof = roc_auc_score(df['T'].values, g_hat)
except Exception:
    auc_oof = np.nan

print(f"Out-of-fold propensity AUC: {auc_oof:.4f}")

# ----------------
# 5) Residuals and DML (residual-on-residual)
# ----------------
rY = df['Y'].values - m_hat
rT = df['T'].values - g_hat

# check mean residuals ~ 0
print(f"mean(rY): {rY.mean():.4e}, mean(rT): {rT.mean():.4e}")

# OLS of rY on rT (no intercept) with robust HC1 SEs
ols_model = sm.OLS(rY, rT.reshape(-1, 1)).fit(cov_type='HC1')
ate_dml = float(ols_model.params[0])
se_dml = float(ols_model.bse[0])
ci_dml = (ate_dml - 1.96 * se_dml, ate_dml + 1.96 * se_dml)

# ----------------
# 6) Naive OLS benchmark (Y ~ T only)
# ----------------
naive = sm.OLS(df['Y'].values, sm.add_constant(df['T'].values)).fit(cov_type='HC1')
ate_naive = float(naive.params[1])
se_naive = float(naive.bse[1])
ci_naive = (ate_naive - 1.96*se_naive, ate_naive + 1.96*se_naive)

# ----------------
# 7) OLS with simple parametric controls (illustrative)
# ----------------
X_for_ols = df[['T'] + colnames[:8]].copy()
X_for_ols = sm.add_constant(X_for_ols)
ols_ctrl = sm.OLS(df['Y'].values, X_for_ols).fit(cov_type='HC1')
ate_ols_ctrl = float(ols_ctrl.params['T'])
se_ols_ctrl = float(ols_ctrl.bse['T'])
ci_ols_ctrl = (ate_ols_ctrl - 1.96*se_ols_ctrl, ate_ols_ctrl + 1.96*se_ols_ctrl)

# ----------------
# 8) Bootstrap ATE CI (resample units, keep X,T,Y together)
# ----------------
rng = np.random.RandomState(SEED)
boot_ates = []
for b in range(BOOTSTRAP_B):
    idx = rng.randint(0, n, n)
    # use cross-fitted rY and rT (we treat them as fixed oof estimates)
    rY_b = rY[idx]
    rT_b = rT[idx]
    try:
        res_b = sm.OLS(rY_b, rT_b.reshape(-1,1)).fit(cov_type='HC1')
        boot_ates.append(float(res_b.params[0]))
    except Exception:
        continue
boot_ates = np.array(boot_ates)
if len(boot_ates) > 0:
    ci_boot = (np.percentile(boot_ates, 2.5), np.percentile(boot_ates, 97.5))
else:
    ci_boot = (np.nan, np.nan)

# ----------------
# 9) Quick CATE summary by X0 quintile
# ----------------
df['rY'] = rY
df['rT'] = rT
df['X0_q'] = pd.qcut(df['X0'], 5, labels=False)
def cate_bin(g):
    denom = (g['rT']**2).sum()
    return (g['rY'] * g['rT']).sum() / denom if denom > 0 else np.nan
cate_by_bin = df.groupby('X0_q').apply(cate_bin).rename('cate_est')

# Also compute pointwise estimated CATE using residual-on-residual slope local approximation:
# for simplicity here use grouping estimate above and also compute simple effect = (rY/rT) clipped
df['cate_pointwise'] = np.where(np.abs(df['rT'])>1e-8, df['rY']/df['rT'], np.nan)

# ----------------
# 10) Important plots: CATE dist, Compare ATEs
# ----------------
plt.figure(figsize=(8,4))
sns.histplot(df['cate_pointwise'].dropna(), bins=50)
plt.title("Pointwise CATE estimates (rY/rT)")
plt.xlabel("CATE (approx)")
plt.tight_layout()
plt.savefig(f"{SAVE_PREFIX}_cate_pointwise_hist.png", dpi=150)
plt.close()

plt.figure(figsize=(6,4))
estimates = pd.Series({
    'DML': ate_dml,
    'Naive OLS': ate_naive,
    'OLS w/ controls': ate_ols_ctrl,
    'True avg tau': df['tau_true'].mean()
})
estimates.plot(kind='bar')
plt.ylabel("ATE estimate")
plt.title("ATE estimates comparison")
plt.tight_layout()
plt.savefig(f"{SAVE_PREFIX}_ate_comparison.png", dpi=150)
plt.close()

# CATE by X0 quintile plot
plt.figure(figsize=(6,4))
cate_by_bin.plot(marker='o')
plt.xlabel("X0 quintile (0..4)")
plt.ylabel("Estimated CATE (bin)")
plt.title("CATE by X0 quintile")
plt.tight_layout()
plt.savefig(f"{SAVE_PREFIX}_cate_by_quintile.png", dpi=150)
plt.close()

# ----------------
# 11) Feature importance summary (average permutation importances across folds)
# ----------------
mean_imp_m = np.mean(np.vstack(model_m_importances), axis=0)
mean_imp_g = np.mean(np.vstack(model_g_importances), axis=0)
imp_df = pd.DataFrame({
    'feature': colnames,
    'imp_m': mean_imp_m,
    'imp_g': mean_imp_g
}).sort_values('imp_m', ascending=False)

imp_df.head(12).to_csv(f"{SAVE_PREFIX}_top_feature_importances.csv", index=False)

# save a combined importance plot for top features
topk = 12
plt.figure(figsize=(8,5))
sns.barplot(x='imp_m', y='feature', data=imp_df.head(topk))
plt.title("Top features by permutation importance (outcome model)")
plt.tight_layout()
plt.savefig(f"{SAVE_PREFIX}_top_features_m.png", dpi=150)
plt.close()

# ----------------
# 12) Print results and save summary
# ----------------
print("---- RESULTS ----")
print("DML ATE:", round(ate_dml, 4), "SE:", round(se_dml, 4), "95% CI (HC1):", (round(ci_dml[0],4), round(ci_dml[1],4)))
print("Bootstrap 95% CI:", (round(ci_boot[0],4), round(ci_boot[1],4)))
print("Naive OLS ATE:", round(ate_naive,4), "95% CI:", (round(ci_naive[0],4), round(ci_naive[1],4)))
print("OLS with controls ATE:", round(ate_ols_ctrl,4), "95% CI:", (round(ci_ols_ctrl[0],4), round(ci_ols_ctrl[1],4)))
print("True sample average tau:", df['tau_true'].mean().round(4))
print("\nCATE by X0 quintile:\n", cate_by_bin.to_string())
print(f"\nSaved summary CSVs and PNGs with prefix: {SAVE_PREFIX}_*")

# Save results and sample
df.sample(n=min(1500, n), random_state=SEED).to_csv(f'{SAVE_PREFIX}_synthetic_sample.csv', index=False)
pd.Series({
    'ate_dml': ate_dml,
    'se_dml': se_dml,
    'ci_dml_low': ci_dml[0],
    'ci_dml_high': ci_dml[1],
    'ci_boot_low': ci_boot[0],
    'ci_boot_high': ci_boot[1],
    'ate_naive': ate_naive,
    'ate_ols_ctrl': ate_ols_ctrl,
    'true_sample_avg_tau': df['tau_true'].mean(),
    'propensity_auc_oof': auc_oof
}).to_csv(f'{SAVE_PREFIX}_results_summary.csv')

# Save cate_by_bin
cate_by_bin.to_csv(f'{SAVE_PREFIX}_cate_by_quintile.csv')

print("\nAll outputs saved. Submit the results CSVs and PNGs to Cultus for best marks.")


Generated dataset n=5000, p=60, mean(T)=0.485
Cross-fitting finished in 114.4s
Out-of-fold propensity AUC: 0.6850
mean(rY): -5.6734e-03, mean(rT): 3.7913e-04


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11864\3267418842.py:210: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cate_by_bin = df.groupby('X0_q').apply(cate_bin).rename('cate_est')


---- RESULTS ----
DML ATE: 1.1939 SE: 0.0346 95% CI (HC1): (1.126, 1.2618)
Bootstrap 95% CI: (np.float64(1.1255), np.float64(1.2776))
Naive OLS ATE: 1.5745 95% CI: (1.4729, 1.6761)
OLS with controls ATE: 1.5517 95% CI: (1.4485, 1.6549)
True sample average tau: 1.1664

CATE by X0 quintile:
 X0_q
0    0.838773
1    1.056887
2    1.233849
3    1.201069
4    1.608024

Saved summary CSVs and PNGs with prefix: dml_run_*

All outputs saved. Submit the results CSVs and PNGs to Cultus for best marks.
